Plot the three longest whiskers from the first frame of the video.

In [ ]:
import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import json

In [ ]:
# Data directory
# data_dir = '/home/wanglab/data/whisker_asym/wa001/test'
# data_dir = '/home/wanglab/data/whisker_asym/sc012/test_full_length'
data_dir = '/home/wanglab/data/whisker_asym/test'
# Load the Parquet file
# parquet_file = 'WA001_080224_01_left.parquet'
# parquet_file = 'sc012_0119_001_left.parquet'
parquet_file = 'sc012_0119_001.parquet'
table = pq.read_table(f'{data_dir}/{parquet_file}')

# Convert the table to a Pandas DataFrame
df = table.to_pandas()
df

In [ ]:
# Filter the DataFrame to include only the first frame (fid = 0)
first_frame_df = df[df['fid'] == 0]

# Get the sides available for the first frame
sides = first_frame_df['face_side'].unique()

# For each side, sort by the "pixel_length" column and take the top 3 longest whiskers
longest_whiskers = []
for side in sides:
    side_df = first_frame_df[first_frame_df['face_side'] == side]
    longest_whiskers.append(side_df.nlargest(3, 'pixel_length'))

In [ ]:
# Read in a frame of the video
# video_name = 'WA001_080224_01_TopCam_10s.mp4'
# video_name = 'sc012_0119_001_20230119-190517_HSCam.avi'
video_name = 'test.mp4'
frame_num = 0
cap = cv2.VideoCapture(f'{data_dir}/{video_name}')
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
ret, frame = cap.read()
# Display the frame
plt.figure(figsize=(8, 6))
# plt.imshow(frame)

In [ ]:
# Create set of colors for up to 20 whiskers, starting with red, green, blue
colors = [(255,0,0), (0,255,0), (0,0,255), (255,255,0), (255,0,255),
            (0,255,255), (128,0,0), (0,128,0), (0,0,128), (128,128,0),
            (128,0,128), (0,128,128), (64,0,0), (0,64,0), (0,0,64),
            (64,64,0), (64,0,64), (0,64,64), (192,0,0), (0,192,0)]

In [ ]:
# Using index of three longest whiskers, plot the whiskers on the frame
for longest_whiskers_side in longest_whiskers:
    for index, whisker_data in longest_whiskers_side.iterrows():
        color_index = index % len(colors)
        color = colors[color_index]
        print(f"Whisker ID: {whisker_data['wid']}, color: {color}")

        print(whisker_data['pixels_x'][0], whisker_data['pixels_y'][0])
        for j in range(whisker_data['pixels_x'].shape[0]):
            # add the image_coord first two values
            x = int(whisker_data['pixels_x'][j])
            y = int(whisker_data['pixels_y'][j])
            cv2.circle(frame, (x, y), 2, color, -1)

In [ ]:
# Plot the frame below the cell in the notebook
plt.imshow(frame)

In [ ]:
# Select the frame with the highest 